# Prerequirements

To be able to run this colab correctly there are two things you need to setup before:

- Make sure you are using a GPU environment, as a CPU wont be able to handle the processing required by the LLM. Due to the size of the model a GPU with at least 16GB of ram is required.

- In the secrets section of the Colab, you must add a HuggingFace key with an account authorized to use Llama3 model of Meta:

    1. You can accept Meta's agreement in [Llama3](https://huggingface.co/meta-llama/Meta-Llama-3-8B)
    2. Once the agreement has been signed, and Meta has authorized you to use their models, you must proceed to obtain a HugginFace api token from the [Api Tokens Page](https://huggingface.co/settings/tokens)
    3. You must set this api token as a secret on this Colab called `HF_TOKEN`, after which you should authorize it to be used on the Colab by checking the blue check.

- This Colab uses Google Drive as a storing place. Thats why the variable defined on the first code execution called `DRIVE_FOLDER` should be fixed to point to the folder where the model will be saved.
    - This folder must have the following structure:
        - `Llama`: This folder will be used to store and retrieve the LLM.
        - `Inputs`: This folder will contain all the json files meant to be processed. This will be a list of posts which each contains a list of comments too.
            - `Inputs/IG`: Folder where the scrapped json files of instagram are.
            - `Inputs/FB`: Folder where the scrapped json files of facebook are.
            - `Inputs/Web`: Folder where the scrapped json files of the web are.
        - `Chunks`: This folder is where the chunks of done processed data will be.
        - `sentiment_output.json`: This will be the final output with the sentiment attached.

# Sentiment Analysis Model

For the model, we will use Huggingface as an interface to easily access a variaty of models. Being allowed to fine tune them and test them in an efficient way.

## Libraries installation and Module Imports

In [ ]:
from google.colab import userdata, drive
drive.mount('/content/drive')
DRIVE_FOLDER="drive/MyDrive/Vero Volley project/SentimentModel"

In [ ]:
!pip install transformers
!pip install datasets
!pip install trl
!pip install peft
!pip install -q accelerate
!pip install -q -i https://pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-m

## Models

For the models, we will mainly focus on LLama3 and Gemma (Meta and Google LLM). This will be trained against a dataset of pre-classified tweet messages for defining if a message is either "positive", "neutral" or "negative".

Other models already pre-tuned are already disponible, but based on Bert (old Google model).

In [ ]:
# Models
model_gemma = "google/gemma-2b"
model_gemma_7b = "google/gemma-7b"
model_llama = "meta-llama/Meta-Llama-3-8B"

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import json

import torch
import torch.nn as nn

import transformers
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb
from trl import SFTTrainer

from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split


## Model Initialization

The model is initialized with specific paramethers for the optimization of the sentiment analysis.

In specific, we load:
- The model with the byte rules.
- The tokenizer with the limit of characters to generate.
- The End-Of-Sequence Token, to define where to stop the text generation.

In [ ]:
model_name = model_llama
tokenizer_model_name = model_name

# Use Pretrained model
model_name = f"./{DRIVE_FOLDER}/Llama"

max_seq_length = 2048
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name, max_seq_length=max_seq_length)
EOS_TOKEN = tokenizer.eos_token

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

## Training & Testing

We start defining the prompt for both training and testing. This defines the task to complete to the LLM, allowing a classification on the specific sentiments.

In [ ]:
def generate_prompt(data_point):
    return (
        f"""generate_prompt
            Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [{data_point["text"]}] = {data_point["label_text"]}
            """.strip()
        + EOS_TOKEN
    )

def generate_test_prompt(data_point):
    text = data_point
    if not isinstance(data_point, str):
        text = data_point["text"]

    return f"""Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [{text}] =""".strip()

### Evaluation

We create the evaluation function, to compare and analyse the difference between the true emotions from the predicted emotions by the model. It defines a performance view of the model.

In [ ]:
def evaluate(y_true, y_pred):
    print()
    labels = ['positive', 'neutral', 'negative']
    mapping = {'positive': 2, 'neutral': 1, 'none':1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

### Parallelization Helper Functions

We define some helper functions to be able to parallelize the prediction of files

#### Parallelization data conversion

For being able to run a big list of files in parallel with multiple colab sessions, we created a new data structure that stores "elements" instead of posts with comments.

An element, is simply either a post or a comment that is meant to be used to predict its sentiment. is composed by the following structure:
```python
{
  "hash": "g23hf..." # Is a hash representation of the post or element, to be interconnected later
  "text": "I dunno, wh..." # The text that needs to be used for prediction
  "sentiment": "negative" # The sentiment, if it was already predicted, if not None
}
```

In [ ]:
def post_to_hash(post):
  return f"{post.get('taken_at_date')}{post.get('date')}{post.get('title')}{post.get('content')}"

def comment_to_hash(post_hash, comment):
  return f"{post_hash}{comment.get('text')}{comment.get('author')}{comment.get('user')}{comment.get('username')}{comment.get('created_at_utc')}{comment.get('date')}"

In [ ]:
def posts_to_elements(posts):
  elements = []
  seen_set = set()

  for post in posts:
    post_hash = post_to_hash(post)
    if post_hash not in seen_set:
      elements.append({
          "hash": post_hash,
          "text": post["title"] if post["title"] else post["content"],
          "sentiment": post["sentiment"] if "sentiment" in post else None
      })
      seen_set.add(post_hash)
      for comment in post["comments"]:
        comment_hash = comment_to_hash(post_hash, comment)
        if comment_hash not in seen_set:
          elements.append({
            "hash": comment_hash,
            "text": comment["text"],
            "sentiment": comment["sentiment"] if "sentiment" in comment else None
          })
          seen_set.add(comment_hash)

  return elements


def elements_to_posts(elements):
  # Platforms to check
  platform_to_separator = {
      "FB": "keyword",
      "IG": "keyword",
      "Web": "keyword"
  }
  # Open posts already analyzed, if not present initialize to empty array
  sentiment_from_element = {}
  seen_set = set()

  for element in elements:
    sentiment_from_element[element["hash"]] = element["sentiment"]


  # Crawl folders to find the files and posts
  posts = []
  platforms = next(os.walk(f'{DRIVE_FOLDER}/Inputs'))[1]
  for platform in platforms:
    if platform not in platform_to_separator:
      continue
    folder_files = next(os.walk(f'{DRIVE_FOLDER}/Inputs/{platform}'))[2]
    folder_files = list(filter(lambda x: (x.split(".")[-1] == "json"), folder_files))

    for filename in folder_files:
      source = filename.split(f"_{platform_to_separator[platform]}")[0]
      with open(f"{DRIVE_FOLDER}/Inputs/{platform}/{filename}", "r") as f:
        data = json.load(f)

      for post in data:
        post["platform"] = platform
        post["source"] = source
        post["title"] = post["title"] if post.get("title") else ""
        post["content"] = post["content"] if post.get("content") else ""
        post["comments"] = post["comments"] if post.get("comments") else []

        post_hash = post_to_hash(post)
        # Skip posts still not analyzed
        if post_hash in sentiment_from_element and post_hash not in seen_set:
          post["sentiment"] = sentiment_from_element[post_hash]
          del sentiment_from_element[post_hash]

          comments = []
          for comment in post["comments"]:
            comment_hash = comment_to_hash(post_hash, comment)

            # Skip comments still not analyzed
            if comment_hash in sentiment_from_element and comment_hash not in seen_set:
              comment["sentiment"] = sentiment_from_element[comment_hash]
              del sentiment_from_element[comment_hash]
              comments.append(comment)
              # seen_set.add(comment_hash)
          post["comments"] = comments

          posts.append(post)
          seen_set.add(post_hash)

  return posts

#### Data Fetching for parallelization

Now, we fetch the data to fetch from the drive, we cross compare with the results already done and get a list of elements "to-do".

This elements are shuffled so there are less probabilities of repeating processing of elements between multiple colab sessions.

In [ ]:
import json
import os
import random

def thousands(number):
  return '{:,}'.format(number).replace(',','.')
def count_elements(posts):
  return len(posts) + sum(map(lambda x: len(x["comments"]), posts))
def count_format_elements(posts):
  return thousands(count_elements(posts))

def merge_chunks():
  set_to_skip = set()

  done_elements = []
  chunk_files = next(os.walk(f'{DRIVE_FOLDER}/Chunks'))[2]
  for chunk_file in chunk_files:
    if "done" not in chunk_file:
      continue

    try:
      with open(f'{DRIVE_FOLDER}/Chunks/{chunk_file}', "r") as f:
        chunk_data = json.load(f)
      for element in chunk_data:
        hash = element["hash"]
        if hash not in set_to_skip:
          done_elements.append(element)
          set_to_skip.add(hash)
    except:
      pass

  return done_elements, set_to_skip




def get_pair_of_data(with_print=True):
  # Platforms to check
  platform_to_separator = {
      "FB": "crawl",
      "IG": "post",
      "Web": "output"
  }
  # Open posts already analyzed, if not present initialize to empty array
  data_done, set_to_skip = merge_chunks()

  # Crawl folders to find the files and posts
  elements_to_process = []
  platforms = next(os.walk(f'{DRIVE_FOLDER}/Inputs'))[1]
  for platform in platforms:
    if platform not in platform_to_separator:
      continue
    folder_files = next(os.walk(f'{DRIVE_FOLDER}/Inputs/{platform}'))[2]
    folder_files = list(filter(lambda x: (x.split(".")[-1] == "json"), folder_files))

    for filename in folder_files:
      source = filename.split(f"_{platform_to_separator[platform]}")[0]
      with open(f"{DRIVE_FOLDER}/Inputs/{platform}/{filename}", "r") as f:
        data = json.load(f)

      for post in data:
        post["platform"] = platform
        post["source"] = source
        post["title"] = post["title"] if post.get("title") else ""
        post["content"] = post["content"] if post.get("content") else ""
        post["comments"] = post["comments"] if post.get("comments") else []

        post_hash = post_to_hash(post)

        # Skip posts already analyzed
        if post_hash not in set_to_skip:
          elements_to_process.append({
              "hash": post_hash,
              "text": post["title"] if post["title"] else post["content"],
              "sentiment": None
          })
          set_to_skip.add(post_hash)

        for comment in post["comments"]:
          comment_hash = comment_to_hash(post_hash, comment)
          if comment_hash not in set_to_skip:
            elements_to_process.append({
                "hash": comment_hash,
                "text": comment["text"],
                "sentiment": None
            })
            set_to_skip.add(comment_hash)

  random.shuffle(elements_to_process)

  # Number of elements to process (posts + comments)
  print("Elements to process:", thousands(len(elements_to_process)))
  print("Elements Skept:", thousands(len(data_done)))
  print()
  return elements_to_process, data_done


### Prediction

We define the prediction function, which uses the model to generate a prediction to a set of messages.

In [ ]:
from google.colab import files
import uuid

"""
  Function for predicting a list of texts with the model
"""
def predict(X_test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**input_ids, max_new_tokens=1, temperature=0.001, pad_token_id = tokenizer.eos_token_id)
        result = tokenizer.decode(outputs[0])
        answer = result.split("=")[-1].lower()
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("non-related")
    return y_pred

"""
  Adapter function to predict a json file of posts and comments
"""
def predict_json(json_data, model, tokenizer, data_to_append=[], chunk_index=0, debug=False):
    # List to collect all texts
    texts = []

    # Iterate over each post and its comments
    json_data = [*json_data]
    for post in json_data:
        # Append the post caption text
        try:
          texts.append(generate_test_prompt(post["title"] if post["title"] else post["content"]))
        except:
          print(post)
          raise Exception(":noooo")
        # Append each comment text
        for comment in post['comments']:
            texts.append(generate_test_prompt(comment["text"]))

    y_pred = []
    if len(texts) > 0:
      # Convert the list of texts into a DataFrame in one go
      df = pd.DataFrame(texts, columns=['text'])
      # Predict emotions
      y_pred = predict(df, model, tokenizer)

    # Assign emotion to posts and comments
    index = 0
    for post in json_data:
        post["sentiment"] = y_pred[index]
        index += 1
        if index < 40 and debug:
            print("-"*30)
            print(f"\n{post['sentiment']}: {post['title'] if post['title'] else post['content']}")
        for comment in post["comments"]:
            comment["sentiment"] = y_pred[index]
            index += 1
            if index < 40 and debug:
                print(f"  {comment['sentiment']}: {comment['text']}")

    checkpoint_filename = f"sentiment_output_[{chunk_index}]_{uuid.uuid4()}.json"
    with open(f"{DRIVE_FOLDER}/Chunks/{checkpoint_filename}", "w") as f:
        json.dump(json_data, f, indent=2)

    # Save output
    for post in data_to_append:
        json_data.append(post)

    with open("sentiment_output.json", "w") as f:
        json.dump(json_data, f, indent=2)
    return json_data

"""
  Function for predicting a list of elements, prebuilt for parallelization
"""
def predict_elements(elements_to_process, model, tokenizer, data_to_append=[], chunk_index=0, debug=False):
    # List to collect all texts
    texts = []

    # Iterate over each element
    elements_to_process = [*elements_to_process]
    for element in elements_to_process:
        texts.append(generate_test_prompt(element["text"]))

    y_pred = []
    if len(texts) > 0:
      # Convert the list of texts into a DataFrame in one go
      df = pd.DataFrame(texts, columns=['text'])
      # Predict emotions
      y_pred = predict(df, model, tokenizer)

    # Assign emotion to posts and comments
    index = 0
    for element in elements_to_process:
        element["sentiment"] = y_pred[index]
        index += 1

    checkpoint_filename = f"done_[{chunk_index}]_{uuid.uuid4()}.json"
    with open(f"{DRIVE_FOLDER}/Chunks/{checkpoint_filename}", "w") as f:
        json.dump(elements_to_process, f, indent=2)

    # Save output
    seen_set = set()
    for element in elements_to_process:
        seen_set.add(element["hash"])
    for element in data_to_append:
        if element["hash"] not in seen_set:
            elements_to_process.append(element)
            seen_set.add(element["hash"])

    return elements_to_process

#### Predict sentiment from list of posts and comments of json

This function cares about the conversion of the list of posts and comments obtained as input from the Drive folder. It converts them into elements, and goes fetching them 500 elements at a time.

On each iteration, it saves the results into chunks of done data, which is further used to avoid these elements on all colab sessions.


In [ ]:
import time

elements_to_process, data_done = get_pair_of_data()

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

elements_to_do = len(elements_to_process)
elements_done = 0
index = 0

STEPS = 500

start_time = time.perf_counter()
# Predict and save output of posts and comments from json
while len(elements_to_process) > 0:
  total_seconds = round(time.perf_counter() - start_time)
  hours = total_seconds // 3600
  minutes = (total_seconds - hours * 3600) // 60
  seconds = (total_seconds - hours * 3600 - minutes * 60)
  percentage_done = (elements_done * 1000 // len(elements_to_process)) / 10

  print(f"Iteration {index}: ({percentage_done}%) {hours}h {minutes}m {seconds}s")

  data_done = predict_json(elements_to_process[:STEPS], model, tokenizer, data_done, index)

  elements_done += STEPS

  print(f"  Done Elements: {thousands(elements_done)}")
  print(f"  Missing Elements: {thousands(elements_to_do - elements_done)}")
  print()

  # Get updated data
  elements_to_process, new_data_done = get_pair_of_data(False)
  seen_set = set()
  for data in data_done:
    seen_set.add(data["hash"])
  for element in new_data_done:
    if element["hash"] not in seen_set:
      data_done.append(element)
      seen_set.add(element["hash"])

  index += 1


Posts: 2.021
Elements to process: 105.643
Posts Skept: 9.420
Elements Skept: 39.541

Iteration 0: (0.0%) 0h 0m 0s
  Using 3 Posts; Elements: 613


100%|██████████| 613/613 [03:11<00:00,  3.21it/s]


  Done 3 Posts; Elements: 613
  Missing 2.018 Posts; Elements: 105.030

Iteration 1: (0.5%) 0h 3m 12s
  Using 3 Posts; Elements: 258


100%|██████████| 258/258 [01:18<00:00,  3.30it/s]


  Done 6 Posts; Elements: 871
  Missing 2.015 Posts; Elements: 104.772

Iteration 2: (0.8%) 0h 4m 31s
  Using 3 Posts; Elements: 30


...

100%|██████████| 105/105 [00:28<00:00,  3.68it/s]


  Done 222 Posts; Elements: 16.182
  Missing 1.799 Posts; Elements: 89.461

Iteration 74: (15.3%) 1h 16m 50s
  Using 3 Posts; Elements: 173


100%|██████████| 173/173 [00:47<00:00,  3.62it/s]


  Done 225 Posts; Elements: 16.355
  Missing 1.796 Posts; Elements: 89.288

Iteration 75: (15.4%) 1h 17m 39s
  Using 3 Posts; Elements: 289


 75%|███████▍  | 216/289 [00:58<00:19,  3.71it/s]

#### Chunk Merging

Once all predictions are done between all colab session, this function will allow to merge all the chunks into a single file, for optimizations of both space and speed.

In [ ]:
def consolidate_chunks():
  total_posts, _ = merge_chunks()
  with open(f"{DRIVE_FOLDER}/Chunks/done.json", "w") as f:
    json.dump(total_posts, f)

# consolidate_chunks()

After the chunks have been consolidated, this line will remove all chunks avoiding the consolidates unified one.

**IMPORTANT!!**: Just run this line once you are sure the consolidation was made correctly, if not the done data will be lost.

In [ ]:
# Clean chunks
# !find  . -name 'sentiment_output_*' -exec rm {} \;

### Training Phase

This section will be focalized on training the model on the first place. Will be composed of an initial testing of the model without training, then a training phase, and at the end a final testing to check on the results of the training.

#### Dataset

For the training and testing, we load the classified dataset of tweets.
This dataset is sampled into equal parts for each emotion, 600 messages per emotion for training and other 600 per emotion for testing. For the validation, we keep 200 messages per emotion.

Is important to note that the dataset includes a bigger quantity of messages, but due to the time requirements and the size of the LLM's used, they had to be restricted to be able to train the model accordingly, reducing training time from 15 hours to just 1.


In [ ]:
df = load_dataset("SetFit/tweet_sentiment_extraction")
df = pd.DataFrame(df["train"])

X_train = list()
X_test = list()
for sentiment in ["positive", "neutral", "negative"]:
    train, test  = train_test_split(df[df.label_text==sentiment],
                                    train_size=600,
                                    test_size=600,
                                    random_state=42)
    X_train.append(train)
    X_test.append(test)

X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)

eval_idx = [idx for idx in df.index if idx not in list(train.index) + list(test.index)]
X_eval = df[df.index.isin(eval_idx)]
X_eval = (X_eval
          .groupby('label_text', group_keys=False)
          .apply(lambda x: x.sample(n=100, random_state=10, replace=True)))
X_train = X_train.reset_index(drop=True)

X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), columns=["text"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1), columns=["text"])

y_true = pd.DataFrame(X_test).label_text
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])


train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

print("Train:")
print(X_train)
print()
print("Test:")
print(X_test)

#### Initial Prediction with Untrained Model

We use the prediction function to get the predicted emotions for the testing dataset.

In [ ]:
y_pred = predict(X_test, model, tokenizer)

100%|██████████| 1800/1800 [23:36<00:00,  1.27it/s]


And we analyse the results of the non-trained model.

In [ ]:
evaluate(y_true, y_pred)

Accuracy: 0.772
Accuracy for label 0: 0.808
Accuracy for label 1: 0.692
Accuracy for label 2: 0.815

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.81      0.80       600
           1       0.68      0.69      0.69       600
           2       0.85      0.81      0.83       600

    accuracy                           0.77      1800
   macro avg       0.77      0.77      0.77      1800
weighted avg       0.77      0.77      0.77      1800


Confusion Matrix:
[[485  99  16]
 [116 415  69]
 [ 18  93 489]]


#### Text

For further increasing the accuracy of the model, we fine-tune it with the training dataset. We define the loss function for the training, the validation function and the training parameters like steps, epochs, etc.

We use the training data for the training process, and the validation data for a testing process of the improvement of the model, to know how it behaves between epochs and keep the weights with the best performance.

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
)

training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=3,
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=False,
    evaluation_strategy='steps',
    load_best_model_at_end=True,
    eval_steps = 112,
    eval_accumulation_steps=1,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    args=training_arguments,
    packing=False,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("trained-model")

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### Final Testing

After training the model, we predict again the emotions of the testing dataset. We analyse the results to be able to compare the improvements of the model in comparison to the untrained model.

In [ ]:
y_pred = predict(X_test, model, tokenizer)

  0%|          | 0/1800 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 1800/1800 [23:37<00:00,  1.27it/s]

Accuracy: 0.700
Accuracy for label 0: 0.758
Accuracy for label 1: 0.592
Accuracy for label 2: 0.750

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.76      0.75       600
           1       0.60      0.59      0.60       600
           2       0.76      0.75      0.75       600

    accuracy                           0.70      1800
   macro avg       0.70      0.70      0.70      1800
weighted avg       0.70      0.70      0.70      1800


Confusion Matrix:
[[455 116  29]
 [131 355 114]
 [ 32 118 450]]


In [ ]:
evaluate(y_true, y_pred)

# Extra space for developing